In [1]:
!pip install transformers
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch

Создание модели

In [4]:

model_name = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Загрузка датасета

In [ ]:
import csv
data = []
with open("data_1.csv") as f:
    reader = csv.reader(f, delimiter=";")
    for i, row in enumerate(reader):
        list_ = "".join(row).split(",")
        list_[1] = int(list_[1])
        data.append(tuple(list_))
        if i > 100000:
            break

data = dict(data)
data_prep = {"text": list(data.keys()), "target": list(data.values())}

In [57]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame.from_dict(data_prep)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=42)

In [ ]:
df.head()

Создание кастомного датасета

In [63]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['text']
        label = self.df.iloc[idx]['target']

        encoding = self.tokenizer(text, padding='max_length', truncation=True, return_tensors='pt', max_length=256)

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }
train_dataset = CustomDataset(train_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

Обучение модели

In [60]:
fp16 = torch.cuda.is_available()
fp16

True

In [64]:
training_args = TrainingArguments(
    output_dir=f'/content/results',
    num_train_epochs=2,
    per_device_train_batch_size=64, # Adjust according conditions
    fp16=fp16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f'/content/logs',
    logging_steps=100,
    do_train=True,
    do_eval=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.02)],
)

In [65]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.016300,0.010373
2,0.007200,0.008572


TrainOutput(global_step=1700, training_loss=0.02677259013933294, metrics={'train_runtime': 842.1428, 'train_samples_per_second': 129.173, 'train_steps_per_second': 2.019, 'total_flos': 7205291263414272.0, 'train_loss': 0.02677259013933294, 'epoch': 2.0})

Сохранение весов

In [67]:
output_model_dir = f'/content/model_config'

model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

('/content/model_config/tokenizer_config.json',
 '/content/model_config/special_tokens_map.json',
 '/content/model_config/vocab.txt',
 '/content/model_config/added_tokens.json',
 '/content/model_config/tokenizer.json')

Оценка модели на тестовом датасете

In [20]:
def classify_personal_data(test_dataloader):
    y_true = torch.tensor([])
    predicted_labels = torch.tensor([])
    model.to("cuda")
    model.eval()
    for inputs in test_dataloader:
        with torch.no_grad():
            outputs = model(**inputs)
        predicted_labels = torch.cat((predicted_labels, torch.argmax(outputs.logits.detach().cpu(), dim=1)))
        y_true = torch.cat((y_true, inputs["labels"].detach().cpu()))
    return predicted_labels, y_true

In [22]:
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred, y_test = classify_personal_data(test_dataloader)


In [26]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, average="macro"))
print('Recall:', recall_score(y_test, y_pred, average="macro"))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

Accuracy: 0.9999504066653442
Precision: 0.9999385900270203
Recall: 0.9941860465116279
F1 score: 0.997028114771274


Инференс

In [40]:
from transformers import BertTokenizer, BertForSequenceClassification

# Function to classify input text
def classify_personal_data(input_text):
    labels = {0: "no data", 1:"personal data", 2: "financial data", 3:"biometrical data"}
    # Tokenize the input text
    # inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")
    inputs = tokenizer(input_text, padding='max_length', truncation=True, return_tensors='pt', max_length=256)
    # Make a prediction
    model.to("cpu")
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(labels[predicted_label])
    return "The input text contains sensitive data." if predicted_label else "The input text does not contain sensitive data."

In [45]:
classify_personal_data("Employee finger")

biometrical data


'The input text contains sensitive data.'